In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "/cache/models/official_models/Llama-2-7b-hf",
)
model = AutoModelForCausalLM.from_pretrained(
    "/cache/models/official_models/Llama-2-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto",
    attn_implementation="sdpa"
)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
input_ids = tokenizer("Plants create energy through a process known as", return_tensors="pt").to("cuda")

output = model.generate(**input_ids, cache_implementation="static")
print(tokenizer.decode(output[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.12/site-packages/torch/_inductor/compile_fx.py:167: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


Plants create energy through a process known as photosynthesis. The process takes place in the leaves of the plant, which are called the green leaves. The green leaves contain chlorophyll, which is the green pigment that absorbs the energy from the sun. The energy is used to make food for the plant.
The leaves of the plant contain chlorophyll, which is the green pigment that absorbs the energy from the sun. The energy is used to make food for the plant.
The chlorophyll in the leaves of the plant is used to make food for the plant. The food is used to make energy for the plant. The energy is used to make food for the plant.
The food is used to make energy for the plant. The energy is used to make food for the plant. The food is used to make energy for the plant. The energy is used to make food for the plant. The food is used to make energy for the plant. The energy is used to make food for the plant. The food is used to make energy for the plant. The energy is used to make food for the 

In [1]:
import numpy as np

# 原始模型 Qwen 的性能
qwen = np.array([24.37, 39.65, 30.98, 44.20, 26.83, 33.39])

# Suppressed-Highest 模型
suppressed_high = np.array([12.58, 4.7, 23.54, 24.07, 0, 19.45])

# Suppressed-Lowest 模型
suppressed_low = np.array([19.63, 11.3, 29.01, 44.51, 6.71, 29.89])

# 计算差值（Qwen - 变种）
diff_high = qwen - suppressed_high
diff_low = qwen - suppressed_low

# 计算平均差距
avg_diff_high = np.mean(diff_high)
avg_diff_low = np.mean(diff_low)

print(f"Suppressed-Highest 平均性能差距: {avg_diff_high:.2f}")
print(f"Suppressed-Lowest 平均性能差距: {avg_diff_low:.2f}")


Suppressed-Highest 平均性能差距: 19.18
Suppressed-Lowest 平均性能差距: 9.73


In [2]:
import numpy as np

# LoKI 模型的性能
loki = np.array([20.53, 37.98, 21.53, 47.04, 24.39, 33.39])

# Global-Highest 模型
global_high = np.array([11.04, 29.34, 10.23, 17.68, 25.00, 26.79])

# Global-Lowest 模型
global_low = np.array([9.83, 30.93, 8.48, 43.09, 24.39, 27.72])

# 计算差距
diff_high = loki - global_high
diff_low = loki - global_low

# 计算平均差距
avg_diff_high = np.mean(diff_high)
avg_diff_low = np.mean(diff_low)

print(f"Global-Highest 平均性能差距: {avg_diff_high:.2f}")
print(f"Global-Lowest 平均性能差距: {avg_diff_low:.2f}")


Global-Highest 平均性能差距: 10.80
Global-Lowest 平均性能差距: 6.74


In [3]:
import numpy as np

# Qwen 原始模型
qwen = np.array([24.37, 39.65, 30.98, 44.20, 26.83, 33.39])

# LoKI(q=20)，跳过参数量字段，仅取6个性能指标
loki_q20 = np.array([19.77, 36.77, 33.16, 47.28, 27.44, 33.27])

# 差距（Qwen - LoKI）
diff = qwen - loki_q20

# 计算平均差距
avg_diff = np.mean(diff)

print(f"LoKI(q=20) 与 Qwen 平均性能差距: {avg_diff:.2f}")


LoKI(q=20) 与 Qwen 平均性能差距: 0.29


In [10]:
import numpy as np
import pandas as pd

# Qwen 性能（跳过 'NaN'）
qwen = np.array([24.37, 39.65, 30.98, 44.20, 26.83, 33.39])

# 模型名称与对应性能（跳过参数量字段）
models = {
    "LoKI(q=5)":   [20.53, 38.36, 30.52, 40.09, 24.39, 34.12],
    "LoKI(q=10)":  [20.53, 37.98, 21.53, 47.04, 24.39, 33.39],
    "LoKI(q=20)":  [19.77, 36.77, 33.16, 47.28, 27.44, 33.27],
    "LoKI(q=30)":  [5.42, 34.04, 0.01, 1.18, 21.34, 29.25],
    "LoRA(r=1)":   [13.12, 5.61, 0.00, 0.24, 0.00, 25.93],
    "LoRA(r=8)":   [0.62, 1.14, 0.00, 0.00, 0.00, 23.20],
    "LB-Reranker": [4.65, 2.65, 0.00, 0.00, 0.00, 23.20]
}

# 计算每个模型相对于 Qwen 的平均下降百分比
results = []
for name, scores in models.items():
    scores = np.array(scores)
    percent_drop = (qwen - scores) / qwen  # 相对下降百分比
    avg_percent_drop = np.mean(percent_drop) * 100  # 转为百分比形式
    results.append((name, avg_percent_drop))

# 转为表格
df = pd.DataFrame(results, columns=["Model", "Avg. % Drop vs Qwen"])
# df = df.sort_values("Avg. % Drop vs Qwen")

print(df.to_string(index=False, formatters={"Avg. % Drop vs Qwen": "{:.2f}%".format}))


      Model Avg. % Drop vs Qwen
  LoKI(q=5)               6.12%
 LoKI(q=10)               8.86%
 LoKI(q=20)               1.70%
 LoKI(q=30)              53.68%
  LoRA(r=1)              75.64%
  LoRA(r=8)              87.52%
LB-Reranker              84.13%


In [11]:
import numpy as np
import pandas as pd

# Llama 作为基准模型（跳过 NaN）
llama = np.array([65.77, 84.5, 73.85, 62.98, 72.6, 80.4])

# 其它模型及其性能（忽略参数量）
models = {
    "LoKI(q=10)": [65.85, 84.99, 75.14, 61.33, 68.90, 77.54],
    "LoKI(q=20)": [65.60, 84.31, 73.60, 60.62, 67.68, 78.18],
    "LoKI(q=30)": [65.49, 84.61, 71.16, 61.09, 70.73, 77.94],
    "ToolACE":    [64.63, 79.53, 46.81, 42.78, 60.37, 72.76]
}

# 计算平均下降百分比
results = []
for name, scores in models.items():
    scores = np.array(scores)
    percent_drop = (llama - scores) / llama
    avg_percent_drop = np.mean(percent_drop) * 100
    results.append((name, avg_percent_drop))

# 结果表格
df = pd.DataFrame(results, columns=["Model", "Avg. % Drop vs Llama"])
df = df.sort_values("Avg. % Drop vs Llama")

print(df.to_string(index=False, formatters={"Avg. % Drop vs Llama": "{:.2f}%".format}))



     Model Avg. % Drop vs Llama
LoKI(q=10)                1.47%
LoKI(q=30)                2.10%
LoKI(q=20)                2.35%
   ToolACE               17.11%


In [1]:
import numpy as np
import pandas as pd

# 基准模型 Qwen 的性能
qwen = np.array([24.37, 39.65, 30.98, 44.20, 26.83, 33.39])

# 变种模型的性能
models = {
    "Suppressed-Highest(q=1)": [12.58, 4.7, 23.54, 24.07, 0, 19.45],
    "Suppressed-Lowest(q=1)":  [19.63, 11.3, 29.01, 44.51, 6.71, 29.89]
}

# 计算平均下降百分比
results = []
for name, scores in models.items():
    scores = np.array(scores)
    percent_drop = (qwen - scores) / qwen
    avg_percent_drop = np.mean(percent_drop) * 100
    results.append((name, avg_percent_drop))

# 转为 DataFrame
df = pd.DataFrame(results, columns=["Model", "Avg. % Drop vs Qwen"])
df = df.sort_values("Avg. % Drop vs Qwen")

# 显示结果
print(df.to_string(index=False, formatters={"Avg. % Drop vs Qwen": "{:.2f}%".format}))


                  Model Avg. % Drop vs Qwen
 Suppressed-Lowest(q=1)              30.35%
Suppressed-Highest(q=1)              57.97%


In [2]:
import numpy as np
import pandas as pd

# 基准模型 Qwen 的性能（对应 6 个任务）
qwen = np.array([24.37, 39.65, 30.98, 44.20, 26.83, 33.39])

# 其它模型及其性能
models = {
    "LoKI(q=10)":            [20.53, 37.98, 21.53, 47.04, 24.39, 33.39],
    "Global-Highest(q=10)":  [11.04, 29.34, 10.23, 17.68, 25.00, 26.79],
    "Global-Lowest(q=10)":   [9.83, 30.93, 8.48, 43.09, 24.39, 27.72]
}

# 计算相对下降百分比
results = []
for name, scores in models.items():
    scores = np.array(scores)
    percent_drop = (qwen - scores) / qwen
    avg_percent_drop = np.mean(percent_drop) * 100
    results.append([name] + list(scores) + [avg_percent_drop])

# 构建 DataFrame
columns = ["Model", "TriviaQA", "GSM8K", "HellaSwag", "WinoGrande", "HumanEval", "IFEval", "Avg(%, ↓)"]
df = pd.DataFrame(results, columns=columns)

# 美化输出
pd.set_option("display.float_format", "{:.2f}".format)
print(df.to_string(index=False, formatters={"Avg(%, ↓)": "{:.2f}%".format}))


               Model  TriviaQA  GSM8K  HellaSwag  WinoGrande  HumanEval  IFEval Avg(%, ↓)
          LoKI(q=10)     20.53  37.98      21.53       47.04      24.39   33.39     8.86%
Global-Highest(q=10)     11.04  29.34      10.23       17.68      25.00   26.79    39.04%
 Global-Lowest(q=10)      9.83  30.93       8.48       43.09      24.39   27.72    30.48%
